In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


In [2]:
!apt-get update
# !apt install chromium-chromedriver
# !wget --no-check-certificate https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,252 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,483 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,351 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,056 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,311 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,748 kB]
Err:13 http://p

In [11]:
# from google.colab import files
# files.download('/content/chromedriver_linux64.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
# !unzip -qq '/content/chromedriver_linux64.zip'

replace chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace LICENSE.chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [13]:
# !which chromedriver

In [ ]:
# https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com/54077842#54077842
# 아래 shell 출처

In [16]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.WPOMY6ZbyA/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.zcRX3INcFC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.gmxyZKIZa1/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://security.ubuntu.com/ubuntu

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os


import warnings
warnings.filterwarnings('ignore')

# name=['비눗방울']
# category=['리뷰']

#비눗방울 리뷰
ns_address="https://search.shopping.naver.com/catalog/32312940336?query=%EB%B9%84%EB%88%97%EB%B0%A9%EC%9A%B8&NaPm=ct%3Dl3wbhe7k%7Cci%3Db1e4a573a939efb90f075e283fc583cf80e1ce6a%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3D74e479a60372823701768de0d2a5983ab4cde64e"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul"
#
service = Service("/usr/bin/chromedriver")

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

d = webdriver.Chrome(service=service, options=options)

#
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# header = {'User-Agent': ''}

# driver_service = Service('/usr/local/bin/chromedriver.exe')

# d = webdriver.Chrome('/usr/bin/chromedriver',options=options) # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element("xpath", shoppingmall_review).click()
sleep(2)

element=d.find_element("xpath", shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

def add_dataframe(stars,reviews,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['star','review'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[stars[i],reviews[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=['null','null']
        n+=1    
    return df1

d.find_element("xpath", shoppingmall_review).click() #스크롤 건드리면 안됨
stars=[]
reviews=[]
cnt=1   #리뷰index
page=1

while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            review=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review.replace('\n',""))
            star=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[1]/span[1]').text
            star = star + "\t"
            stars.append(star.strip("평점"))
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료
        
    else : 
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a[11]').click()
            else : next_page=d.find_element("xpath", '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[4]/div[3]/a['+str(page%10+2)+']').click()
            
        except: break
            



페이지 1 

1 애기랑 사이좋게 하려고 두개 샀어요 😁
방울이 엄청 많이 나와요!!!
액도 좋은건지 쿠x에서 산 비눗방울 액은
방울이 아예 안만들아졌었는데 ! 😡아오
이건 속 시원히 아주 빵빵 많이 나오네요!!!
! 👍
거기다 불 때 애가 실수로 들이켜도
! !
이거랑 비슷하게 생긴거 과거에 샀었는데
그건 애가 실수로 마시면 입안 가득 거품이였는데😰
요건 그런거 1도 없어서 !!
그리고 라이브 때 엄청난 행운 주셔서 감사해여❤️‍🔥
넘 ㅎㅎ
알라뷰 아가드👍❤️‍🔥 평점5	 

2 요즘 날이 좋아서 외출하면 아기한테 비누방울 불어주고 싶어 구매했어요. 역류방지 기능이 있어 아기가 직접 불면서 놀면 더 좋겠지만 아직 어려서 엄마가 아날로그ㅎㅎ 어린시절 추억 떠올리며 마구마구 불어줬습니다ㅎㅎ 아기자기 작은 비눗방울이 엄청 풍성하게 나와서 사진 찍기 좋고👍 쉽게 터지지 않아서 더 좋은 것 같아요! 화창한 날 아가드 비눗방울로 행복한 시간 보냈어요💓 평점5	 

3 아가 기념일 사진 셀프촬영할때마다 분위기용+포토용으로 구매했는데, 사진 촬영하면서도 예쁜 분위기 샤샤샥➰ 연출하면서 찍을 수 있어요😚🤍✨ 아가드 비눗방울 덕분에 홈스튜디오 촬영 매번 대성공👼🏻🤍💛✨✨ 촬영용 제외하고도 놀이용 또한 말모〰️ 우리 사랑둥이 아가가 아가드 비눗방울 놀이 너무 좋아해용👼🏻 선물용으로 하나 더 구매했었는데, 추가 구매 또한 계속 계속 하고 싶을 정도로 소장가치 충분해용👏🏻👏🏻💛🤍✨✨ 평점5	 

4 17개월 아기가 정말정말 좋아합니다. 갖고노는 사진을 찍은줄알았는데 없어요. 안찍었나보네요ㅋㅋ 세워두고 받침에 부어두고 쓰는건 불편한데 모양이 특이하고 불기 쉽게 되있어서 좋네요.
나가서 하기엔 손에 많이 묻어서 못 할거같구요, 그래도 애기가 비눗방울 부는곳을 먹으려고하고 입술이랑 띄어서 부는걸 잘못하는데 이건 나팔같이생겨서 괜찮네요. 근데 큰단점은 후를 못하고 반대로 빨면 입안으로 들어와서 먹을까봐 아직 주진 못하겠어요.
혼자 불어보라고 주는건 조금 더 큰다음 주던지 해야될거같습니다.

In [18]:
df4=add_dataframe(stars,reviews,cnt)
#save()
df4 = df4['review']

df4.to_csv('reviewCrawling.txt', header=False, index=False)